In [1]:
using OMEinsum

eincode = EinCode([[1, 2], [2, 3], [3, 4], [4, 1], [1], [2], [3], [4]], Int[])
size_dict = uniformsize(eincode, 2)

contraction_complexity(eincode, size_dict)

Time complexity: 2^4.0
Space complexity: 2^0.0
Read-write complexity: 2^4.643856189774725

In [2]:
optcode = optimize_code(eincode, size_dict, TreeSA())

SlicedEinsum{Int64, DynamicNestedEinsum{Int64}}(Int64[], 1, 1 -> 
├─ 4∘1, 1∘4 -> 1
│  ├─ 4∘2, 1∘2 -> 4∘1
│  │  ├─ 2, 2∘4 -> 4∘2
│  │  │  ├─ 2
│  │  │  └─ 2∘3, 3∘4 -> 2∘4
│  │  │     ├─ 3, 2∘3 -> 2∘3
│  │  │     │  ⋮
│  │  │     │  
│  │  │     └─ 3∘4
│  │  └─ 1∘2
│  └─ 4∘1, 4 -> 1∘4
│     ├─ 4∘1
│     └─ 4
└─ 1
)

In [3]:
contraction_complexity(optcode, size_dict)

Time complexity: 2^5.087462841250339
Space complexity: 2^2.0
Read-write complexity: 2^6.108524456778169

In [4]:
# greedy optimizer

greedy = GreedyMethod(
    α = 0.0,                # the parameter of the cost function
    temperature = 0.0,      # the temperature of the hyper-greedy method
    nrepeat = 10,           # the number of trials
)

GreedyMethod{Float64, Float64}(0.0, 0.0, 10)

In [5]:
# treesa optimizer

treesa = TreeSA(
    sc_target = 20,          # the target space complexity
    βs = 0.01:0.05:15,       # the inverse temperatures
    ntrials = 10,            # the number of trials
    niters = 50,             # the number of iterations at each temperature
    sc_weight = 1.0,         # the relative weight of the space complexity
    rw_weight = 0.2,         # the relative weight of the read-write complexity
    initializer = :greedy,   # the initializer of the contraction order
    nslices = 0,             # the number of sliced indices
    fixed_slices = Any[],    # the manually fixed sliced indices
    greedy_config = GreedyMethod(0.0, 0.0, 1), # the method used as initializer
)

TreeSA{Int64, StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}, GreedyMethod{Float64, Float64}, Any}(20, 0.01:0.05:14.96, 10, 50, 1.0, 0.2, :greedy, 0, Any[], GreedyMethod{Float64, Float64}(0.0, 0.0, 1))

In [6]:
# KaHyPar Binary partition

using KaHyPar

kahyparbiparatite = KaHyParBipartite(
    sc_target = 25,               # the target space complexity
    imbalances = 0.0:0.005:0.8,   # the imbalances of the partition
    max_group_size = 40,          # the maximum size of the partition
    sub_optimizer = GreedyMethod(0.0, 0.0, 10),  # the sub-optimizer
)

KaHyParBipartite{Int64, StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}, GreedyMethod{Float64, Float64}}(25, 0.0:0.005:0.8, 40, GreedyMethod{Float64, Float64}(0.0, 0.0, 10))

In [7]:
# SA Binary partition

sabiparatite = SABipartite(
    sc_target = 25,          # the target space complexity
    ntrials = 50,            # the number of trials
    βs = 0.1:0.2:14.9,       # the inverse temperatures
    niters = 1000,           # the number of iterations at each temperature
    max_group_size = 40,     # the maximum size of the partition
    sub_optimizer = GreedyMethod(0.0, 0.0, 10),   # the sub-optimizer
    initializer = :random,   # the initializer of the contraction order
)

SABipartite{Int64, StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}, GreedyMethod{Float64, Float64}}(25, 50, 0.1:0.2:14.9, 1000, 40, GreedyMethod{Float64, Float64}(0.0, 0.0, 10), :random)

In [8]:
# exact treewidth

exacttreewidth = ExactTreewidth(
    greedy_config = GreedyMethod(nrepeat = 1),  # the sub-optimizer
)

ExactTreewidth{GreedyMethod{Float64, Float64}}(CliqueTrees.SafeRules{CliqueTrees.MMW, CliqueTrees.BT}(CliqueTrees.MMW(), CliqueTrees.BT()), GreedyMethod{Float64, Float64}(0.0, 0.0, 1))